![ERO](img/logo.png)

**OpenTRNG** is an open-source framework for developing, and testing **Physical True Random Number Generators** (PTRNG). This project is dedicated to delivering the community open-source implementations of reference ring-oscillator based TRNGs. With **OpenTRNG**, you have the ability to:

1. Emulate RO's physical noise
2. Emulate Raw Random Numbers
3. Simulate and run the PTRNG on hardware plateforms
4. Analyze and evaluate the random outcomes

Links:
* Site: https://opentrng.org
* GitHub: https://github.com/opentrng/ptrng

# Introduction to TRNG

💡 What makes up a **Physical True Random Number Generator**?

It starts with an **analog physical phenomenon**, which is sampled to digital, producing the **Raw Random Number** (RRN). This raw output undergoes **algorithmic post-processing** (the conditioner) to refine it into the **Intermediate Random Number** (IRN). Additionally, **online tests** monitor the quality of randomness, while a **total failure alarm** ensures reliability.

![PTRNG](img/ptrng.png)

This structure aligns with the **PTG.2** functionality class of **BSI AIS 20/31** and **NIST SP 800-22 90B** requirements, meeting stringent standards for secure random number generators.

💥 Random bits for TRNGs come from 𝗽𝗵𝘆𝘀𝗶𝗰𝗮𝗹 𝗻𝗼𝗶𝘀𝗲 sources.

1. **Thermal noise**, uncorrelated and inherently unpredictable, remains the ideal entropy source.
2. However, as technology node shrinks, **flicker noise** becomes more prominent

Flicker noise is correlated over short periods but, with adequate conditions, can still be leveraged to produce reliable random bits.


# Python imports

In [ ]:
import sys
sys.path.append('../analysis')
sys.path.append('../emulator')
sys.path.append('../remote')
import numpy as np
import matplotlib.pyplot as plt
import emulator
import entropy
import binutils
import allanvariance
import lsne
import fluart
import regmap
import frequency

# Emulate noisy ring-oscillators

🔁 Ring oscillators consist of a series of inverters arranged to produce a periodic signal that inherently includes phase noise, commonly referred to as jitter.

![Ring-Oscillator](img/ringo.png)

Generate **10,000,000** periods for two ring-oscillators at **99MHz** and **98MHz** with noise mix as measured in **Xilinx Artix-7** FPGA at 100MHz

In [ ]:
ro0 = emulator.generate_periods(emulator.GENPERIODS, 99e6, emulator.A1_F100M, emulator.A2_F100M)
ro1 = emulator.generate_periods(emulator.GENPERIODS, 98e6, emulator.A1_F100M, emulator.A2_F100M)

In [ ]:
ro0[-3:]

Plot **period distribution** for both ROs and display mean value as well as its standard deviation

In [ ]:
plt.hist(ro0[1:], bins=1000, label='RO0', alpha=0.75)
plt.hist(ro1[1:], bins=1000, label='RO1', alpha=0.75)
plt.legend()
plt.title("Emulated RO period distribution")
plt.xlabel("RO period (s)")
plt.ylabel("Occurrence")
print("RO0: μ={:} σ={:}".format(np.mean(ro0), np.std(ro0)))
print("RO1: μ={:} σ={:}".format(np.mean(ro1), np.std(ro1)))

# Emulate Raw Random Numbers

❓ **How do we create true randomness in hardware**? Sampling the phase noise of ring oscillators is a strong candidate for the physical entropy source in a TRNG. By **sampling a free running ring-oscillator** with an unsynchronized clock, random bits can be generated.

This technique leverages the inherent **unpredictability of thermal and flicker noise in transistors**, providing a robust foundation for secure random number generation.

## Elementary Ring-Oscillator

🌌 The **Elementary Ring-Oscillator** (ERO) is straightforward to implement but has a slower output rate.

![ERO](img/ero.png)

As we are samplling unsynchronized signal with each other, Electronic Design Automation (EDA) tools cannot garantee timing constraints. We have to take into account the event of sampling while data (RO1) is changing.

![Setup and hold times](img/setuphold.png)

**Setup and hold** times as in Xilinx Artix-7 datasheet

In [ ]:
ts = 0.11e-9
th = 0.18e-9

**Generate random bits** with the ERO, in order to accumulate enough noise we use a division factor of 1000

In [ ]:
div = 1000
bits, valid, resolved = emulator.ero(div, ro0, ro1, ts, th)

Plot the first 100 generated **Raw Random Numbers** (bits) and display the bias

In [ ]:
plt.step(range(100), bits[:100])
plt.title("Emulated ERO raw random bits")
plt.xlabel("Samples")
plt.ylabel("Binary value")
plt.yticks([0, 1])
print("Bias: {:}".format(np.mean(bits)))

Estimate the **entropy** of the generated bitstream

In [ ]:
print("Shannon entropy:          {:}".format(entropy.shannon(bits)))
print("Markov entropy estimator: {:}".format(entropy.markov(bits)))
print("MCV entropy estimator:    {:}".format(entropy.mcv(bits)))

Ratio of bit sampled while **valid timing** constraints

In [ ]:
np.sum(valid)/len(bits)

## Multi Ring-Oscillators

🚩 The **Multi Ring-Oscillators** (MURO) delivers higher entropy throughput and output rates but introduces greater design complexity and a potential risk of locking.

![MURO](img/muro.png)

For the MURO (t=4) we replace RO1 with multiple ring-oscillators, here **4 ROs from 96MHz to 99MHz**

In [ ]:
fx = [96e6, 97e6, 98e6, 99e6]
t = len(fx)
rox = np.empty((0, emulator.GENPERIODS))
for f in fx[1:]:
    ro = emulator.generate_periods(emulator.GENPERIODS, f, emulator.A1_F100M, emulator.A2_F100M)
    rox = np.vstack((rox, ro))

**Generate** the random bits

In [ ]:
bits, valid, resolved = emulator.muro(div, rox[0], rox[1:], ts, th)

Plot the first 100 generated **Raw Random Numbers** (bits) and display the bias

In [ ]:
plt.step(range(100), bits[:100])
plt.title("Emulated MURO (t={:d}) Raw random bits".format(t))
plt.xlabel("Samples")
plt.ylabel("Binary value")
plt.yticks([0, 1])
print("Bias: {:}".format(np.mean(bits)))

Estimate the **entropy** of the generated bitstream

In [ ]:
print("Shannon entropy:          {:}".format(entropy.shannon(bits)))
print("Markov entropy estimator: {:}".format(entropy.markov(bits)))
print("MCV entropy estimator:    {:}".format(entropy.mcv(bits)))

Ratio of bit sampled while **valid timing** constraints

In [ ]:
np.sum((2-valid)%2 == 0)/len(bits)

## Coherent Sampling Ring-Oscillator

🚀 Meanwhile, the **Coherent Sampling Ring-Oscillator** (COSO) stands out with its high output rate, intrinsic alarms, and built-in online entropy monitoring.

![COSO](img/coso.png)

Generate COSO **raw counter** values

In [ ]:
counter = emulator.coso(ro0, ro1)

Plot the counter **distribution** and measure their mean value

In [ ]:
plt.hist(counter, bins=np.max(counter))
plt.title("Emulated COSO raw counter distribution")
plt.xlabel("Counter values")
plt.ylabel("Occurrence")
print("Mean counter value: {:}".format(np.mean(counter)))

Extract the **Raw Random Numbers** (bits) by selecting the less significant bit (LSB)

In [ ]:
bits = counter % 2

Plot the first 100 generated **RRN** and display the bias

In [ ]:
limit = 100
plt.step(range(limit), bits[:limit])
plt.title("Emulated COSO Raw Random bits")
plt.xlabel("Samples")
plt.ylabel("Binary value")
plt.yticks([0, 1])
print("Bias: {:}".format(np.mean(bits)))

# HDL simulation

🟢🟡🔴 OpenTRNG includes hardware description files written in **VHDL** along with dedicated **testbenches** for validation. The testbenches are written in Python using the *Cocotb* framework, ensuring a flexible verification process (*Cocotb* is language and simulator agnostic).

Additionally, the RRN emulators act as a **golden reference model** for HDL simulations, providing a *bit-true* baseline for comparison.

![HDL simulation](img/simulation.png)

Example of COSO digital design **simulation** output test cases

```
********************************************************************
** TEST                                     STATUS  SIM TIME (ns) **
********************************************************************
** test_ptrng.test_coso_onlinetest_valid     PASS       11510.00  **
** test_ptrng.test_coso_conditioning         PASS        1380.00  **
** test_ptrng.test_coso_onlinetest_invalid   PASS       11500.00  **
** test_ptrng.test_coso_alarm                PASS        1000.00  **
********************************************************************
** TESTS=4 PASS=4 FAIL=0 SKIP=0                         25390.00  **
********************************************************************
```

Unfortunately, no demo possible in Jupyter notebook 😉

# Hardware

## Generate random bits with the COSO

👉 Let's have a **demo** of OpenTRNG running the **COSO** entropy source on a Xilinx Artix-7 **FPGA** demo board!

OpenTRNG provides all the essential hardware files needed to interface the PTRNG with a remote computer. It enables full control of the TRNG through configuration and status registers while allowing retrieval of generated random bitstreams (or RRN/IRN) from the hardware.

![Hardware top-level](img/toplevel.png)

Open the **connection** to OpenTRNG remote device

In [ ]:
interface = fluart.CmdProc()
trng = regmap.RegMap(interface)

**Configure** the TRNG registers for COSO operation

In [ ]:
trng.control_bf.reset = 1        # Reset the TRNG
trng.control_bf.conditioning = 0 # No conditionning, access to RRN
trng.fifoctrl_bf.nopacking = 0   # Activate bit backing for reading binary stream
trng.ring_bf.en = 0x3            # Enable RO0 and RO1
trng.fifoctrl_bf.clear = 1       # Clear the FIFO buffer
burstsize = trng.fifoctrl_bf.burstsize

Measure RO0 and RO1 **frequency**

In [ ]:
trng.freqctrl_bf.en = 1
f0 = frequency.read(trng, 0)
f1 = frequency.read(trng, 1)
trng.freqctrl_bf.en = 0
print("RO0: {:f} MHz".format(f0))
print("RO1: {:f} MHz".format(f1))

Read **1Mbit of RRN** generated by the COSO

In [ ]:
bytes = []
while len(bytes)*8 < 1_000_000:
	bytes += interface.burstread(trng.FIFODATA_ADDR, burstsize)
bits = binutils.to_bits(bytes)

Plot the first 100 generated **RRN** (bits) and display the bias

In [ ]:
limit = 100
plt.step(range(limit), bits[:limit])
plt.title("Hardware COSO raw random bits")
plt.xlabel("Samples")
plt.ylabel("Binary value")
plt.yticks([0, 1])
print("Bias: {:}".format(np.mean(bits)))

Estimate generated **entropy**

In [ ]:
print("Shannon entropy:          {:}".format(entropy.shannon(bits)))
print("Markov entropy estimator: {:}".format(entropy.markov(bits)))
print("MCV entropy estimator:    {:}".format(entropy.mcv(bits)))

## Extraction of noise parameters

Reset the COSO and read **100,000 raw counter values** from the device

In [ ]:
trng.control_bf.reset = 1        # Reset the TRNG
trng.control_bf.conditioning = 0 # No conditionning, access to RRN
trng.fifoctrl_bf.nopacking = 1   # Disable bit backing for reading counter values
trng.ring_bf.en = 0x3            # Enable RO0 and RO1
trng.fifoctrl_bf.clear = 1       # Clear the FIFO buffer

Read **100,000 Raw counter values** from the COSO device

In [ ]:
bytes = []
while len(bytes)/4 < 100_000:
	bytes += interface.burstread(trng.FIFODATA_ADDR, burstsize)
counter = [int.from_bytes(bytes[i:i+4], 'big') for i in range(0, len(bytes), 4)]

Plot the COSO counter **distribution**

In [ ]:
plt.hist(counter, bins=50)
plt.title("Hardware COSO Raw counter distribution")
plt.xlabel("Counter values")
plt.ylabel("Occurrence")
print("Mean counter value: {:}".format(np.mean(counter)))

Extract the **noise coefficients** for the thermal and flicker noise read from the COSO counter values by computing its **Allan's Variance**

In [ ]:
# Compute Allan's variance
nspace, allanvar = allanvariance.compute(counter)

# Do the polynomial regression LSNE (np.polyfit only fit values in high decades)
poly = lsne.regression(nspace, allanvar, [2, 1, 0])
print("Extracted coefficients:")
print(" - thermal: {:e}".format(poly[1]))
print(" - flicker: {:e}".format(poly[0]))

# Plot in log/log
plt.title("Noise thermal/flicker noise mix\n (Xilinx Artix 7 at 100 MHz)")
plt.xscale('log')
plt.yscale('log')
plt.xlabel('N (accumulation)')
plt.ylabel('Normalized variance')
plt.grid(visible=True, which='major', axis='both')
plt.plot(nspace, allanvar, marker='+')
plt.plot(nspace, np.polyval(poly, nspace), color='red')
plt.legend(['Variance','Polynomial fit'])

# Conclusion

👉 The OpenTRNG framework streamlines the development and testing of Physical **True Random Number Generators** (PTRNGs).

* The **emulators** accurately model thermal and flicker noise as they appear in the targeted hardware, ensuring realistic randomness generation.
* The provided HDL **implementations** can be simulated, verified against the RRN golden model from the emulator, and deployed on FPGA or ASIC platforms.
* **Validation tools** assess entropy levels and estimate noise coefficients, enabling precise characterization of components across different technology nodes.

![OpenTRNG work-flow](img/workflow.png)

🙏 Feel free to test, use and improve it!

http://opentrng.org